## Quant Analyst Interview

#### 1. Given “Employee” table below:

| id | Name | Salary | manager_id |
|----|------|--------|------------|
| 1 | John  | 300    | 3 |
| 2 | Mike  | 200    | 3 |
| 3 | Sally | 550    | 4 |
| 4 | Jane  | 500    | 7 |
| 5 | Joe   | 600    | 7 |
| 6 | Dan   | 600    | 3 |
| 7 | Phil  | 550    | NULL |
|...|  ...  |  ...   |...|

  - Give the name of employees, whose salaries are greater than their immediate
manager’s
  - What is the average salary of employees who do not manage anyone? In the sample
above, that would be John, Mike, Joe and Dan, since they do not have anyone
reporting to them

We can answer both of these using SQL query

> Give the name of employees, whose salaries are greater than their immediate manager’s

```
SELECT e2.name
FROM employee e1
JOIN employee e2
ON e1.id = e2.manager_id
WHERE e2.salary > e1.salary;
```

_Reasoning_: We do a self-join to get a list of entries on the condition which matches employee's id to their manager's id. Now the "second" table is the one that has list of employees, that's the table we are interested in. Then we compare whose salary from that table is higher than the employees in "first" table, which contains the details of their managers. For the table we get the result: `Dan | Sally | Joe`

> What is the average salary of employees who do not manage anyone?

```
SELECT AVG(salary)
FROM employee
WHERE name NOT IN (
	SELECT e1.name
	FROM employee e1
	JOIN employee e2
	ON e1.id = e2.manager_id)
```

_Reasoning_: We use a similar approach as before. Self join the table on id and manager_id, but this time to get a list of employees from "first" table i.e. people who are managers. Then we use it as a subquery to get the list of names that aren't anyone's manager. Once we have that, we can simply take an average of the salary, which in this case is `425.00`.

#### 2. Write a function ‘exists’ which takes a variable symbol v and returns whether v is defined.

In python, it is quite straight-forward to check if a variable exists using an `if` statement. However, when we use a function to check if a variable exists or not, we need to specify if it is _local_ or _global_ variable. From the context of this question, we can assume that we are looking for a global variable. In that case, the `exists()` function can be defined as below. The function returns a boolean True/False as result.

_Note_: Since we are checking for global variable we need to make sure we pass the name of the variable as argument because the function `globals()` returns a dictionary of all global varibales

In [1]:
# function to check if a varible exists
def exists(v):
    # check if the variable exists in the globals
    if v in globals():
        # if it exists, return True
        return True
    else:
        # else, return False
        return False

In [2]:
exists('v')

False

In [3]:
v = 5
exists('v')

True

#### 3. Create a function to compute N layer of a Pascal Triangle. The first 4 later will looks like:
1

1 1

1 2 1

1 3 3 1

We will write a python function to create the Pascal Triangle, which takes `N` as input and returns the pascal triangle of `N` layers.

In [4]:
# function to print pascal's triangle
def print_pascal_tri(lst):
    for row in lst:
        print(row)

In [5]:
# function to create pascal triangle of N layers
def pascal_triangle(N):
    # instantiate an empty list to store the layers
    pascal_tri_list = []
    
    # run a for loop for N number of times
    for i in range(N):
        # list of numbers in each layer
        layer = []
        
        if not pascal_tri_list: # if the full list is empty, append 1
            layer.append(1)
        else: # else insert the intermediate numbers
            # get the length of the previous layer
            prev_layer_len = len(pascal_tri_list[i-1])
            
            # run a second loop one more than previous layer's length
            for j in range(prev_layer_len + 1):
                # for the first two layers just add 1s
                if j == 0 or j == 1:
                    layer.append(1)
                else:
                    # insert the sum of top two numbers at previous index
                    layer.insert(j-1, pascal_tri_list[i-1][j-2] + pascal_tri_list[i-1][j-1])
                    
        # append the layer to final list
        pascal_tri_list.append(layer)
    
    # print the pascal's triangle
    print_pascal_tri(pascal_tri_list)

In [6]:
pascal_triangle(5)

[1]
[1, 1]
[1, 2, 1]
[1, 3, 3, 1]
[1, 4, 6, 4, 1]


_Reasoning_: The comments should be enough to follow along. Basically, we are running a for loop, and at every step checking the previous layer to update the current layer. At the end, in python, it is a list of lists and we print the results. This is a brute force of getting the result and am sure there's an efficient way to solve this by taking advantage of the fact that Pascal's Triangle is an arrangement of binomial coefficients. Using multiple for loops and lists is neither memory nor compute efficient.

#### 4. Assume have the following portfolio as of 2016/01/01:
|      |   |
|------|---|
|AAPL.O|15%|
|IBM.N |20%|
|GOOG.O|20%|
|BP.N  |15%|
|XOM.N |10%|
|COST.O|15%|
|GS.N  |5% |

  - Using historical daily returns (Yahoo/Google Finance or any other market data
source), calculate VaR95% and CVaR95% of the portfolio as of 2016/12/31
  - Using expected mean, covariance matrix and parametric method, calculate VaR95%
and CVaR95%
  - Assume you can change weights, allow shorting but no leverage (i.e. sum of weights
equal 100%), and rebalance monthly. What is the optimal portfolio holding by end of
each month till end of 2016

_Notes_: If you have other assumption(s) please state clearly

First, downloaded some historical daily stock data from [Yahoo finance](https://finance.yahoo.com/) for all the stocks in the portfolio. Saved them as text csv files in the _datesets_ folder.

In [7]:
# import the libraries
import pandas as pd
import numpy as np

In [8]:
# import all the stock data as pandas dataframe
aapl = pd.read_csv('datasets/AAPL.csv')
ibm = pd.read_csv('datasets/IBM.csv')
goog = pd.read_csv('datasets/GOOG.csv')
bp = pd.read_csv('datasets/BP.csv')
xom = pd.read_csv('datasets/XOM.csv')
cost = pd.read_csv('datasets/COST.csv')
gs = pd.read_csv('datasets/GS.csv')

In [9]:
# list of all stocks in portfolio
port_list = ['AAPL', 'IBM', 'GOOG', 'BP', 'XOM', 'COST', 'GS']

In [10]:
# our portfolio
port = pd.DataFrame([0.15, 0.20, 0.20, 0.15, 0.10, 0.15, 0.05],
                    index=port_list,
                    columns=['Weights']).T
port

,AAPL,IBM,GOOG,BP,XOM,COST,GS
Weights,0.15,0.2,0.2,0.15,0.1,0.15,0.05


In [11]:
# function to add the return column to the dataframes
def add_return(df):
    df['Return'] = df['Adj Close'].pct_change()

In [12]:
# call the function on all individual stock dataframes
add_return(aapl)
add_return(ibm)
add_return(goog)
add_return(bp)
add_return(xom)
add_return(cost)
add_return(gs)

In [13]:
# Create a stocks dataframe with the returns for all stocks
stocks = pd.DataFrame([aapl.Return, ibm.Return, goog.Return, bp.Return, xom.Return, cost.Return, gs.Return],
                      index=port_list).T
stocks.set_index(aapl.Date, inplace=True)
stocks

,AAPL,IBM,GOOG,BP,XOM,COST,GS
Date,,,,,,,
2016-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,-0.025059,-0.000735,0.000998,-0.004185,0.008521,0.002444,-0.017219
2016-01-06,-0.019570,-0.005006,0.001400,-0.020369,-0.008321,-0.009254,-0.024412
2016-01-07,-0.042205,-0.017090,-0.023170,-0.028713,-0.016006,-0.022909,-0.030735
2016-01-08,0.005288,-0.009258,-0.016410,-0.017669,-0.020202,-0.017504,-0.004131
...,...,...,...,...,...,...,...
2016-12-23,0.001978,-0.002095,-0.001706,0.002981,-0.001761,0.000062,0.003540
2016-12-27,0.006351,0.002579,0.002076,0.002431,0.000441,-0.000185,0.002448
2016-12-28,-0.004264,-0.005684,-0.008212,0.003772,-0.004959,-0.006418,-0.003767


> Using historical daily returns (Yahoo/Google Finance or any other market data source), calculate VaR95% and CVaR95% of the portfolio as of 2016/12/31

Value-at-Risk aka VaR is a risk evaluation for an investment over a period of time calculated as a loss in a certain confidence interval. VaR95%, in this case, can mean the value with 5% risk of loss.

Conditional-Value-at-Risk is an average loss when the VaR is exceeded. Similarly CVaR95% would mean the average value with 5% risk of loss after the VaR has exceeded.

Using historical data, VaR is calculated from the returns of each stock. It is the lower 5% quartile (0.05 quantile = 5% quartile) of the returns. Taking the average of all the returns below 5% quartile gives the CVaR.

_Note_: This method assumes normal distribution.

In [14]:
# get the 5% quartile for stock returns
stocks.quantile(0.05)

AAPL   -0.023107
IBM    -0.018298
GOOG   -0.020876
BP     -0.029093
XOM    -0.019518
COST   -0.017777
GS     -0.025022
Name: 0.05, dtype: float64

To calculate the VaR of the portfolio we multiply the respective stock with the stock weight in our portfolio and add them up to get the total VaR.

In [15]:
# array of values of 5% quartile of stock returns
stocks.quantile(0.05).values

array([-0.0231074 , -0.01829782, -0.02087635, -0.02909263, -0.01951797,
       -0.01777668, -0.02502246])

In [16]:
# array of values of stock weights in our portfolio
port.values[0]

array([0.15, 0.2 , 0.2 , 0.15, 0.1 , 0.15, 0.05])

We need to multiply these two vectors and add them together. Mathematically, this is a dot product.

In [17]:
# perform dot product on the above two arrays to get VaR95%
var95_hist = port.values[0].dot(stocks.quantile(0.05).values)
var95_hist

-0.02153425940085934

So, the Value-at-Risk with 5% probability of loss is 2.15%. Next, for CVaR, we need to calculate average of all the returns in the lower 5% quartile.

In [18]:
# average of all returns below 5% stock returns quartile
stocks[stocks.apply(lambda x: x < x.quantile(0.05))].mean()

AAPL   -0.033328
IBM    -0.029571
GOOG   -0.031524
BP     -0.041237
XOM    -0.025815
COST   -0.023700
GS     -0.038277
dtype: float64

To calculate CVaR of the portfolio we multiply the respective stock with the stock weight in our portfolio and add them up to get the total CVaR.

In [19]:
# array of values of average of 5% quartile of stock returns
stocks[stocks.apply(lambda x: x < x.quantile(0.05))].mean().values

array([-0.03332801, -0.02957119, -0.03152437, -0.0412372 , -0.02581501,
       -0.02370047, -0.03827725])

In [20]:
# array of values of stock weights in our portfolio
port.values[0]

array([0.15, 0.2 , 0.2 , 0.15, 0.1 , 0.15, 0.05])

Similar to before, we need to multiply these two vectors and add them together. Mathematically, this is a dot product.

In [21]:
# perform dot product on the above two arrays to get CVaR95%
cvar95_hist = port.values[0].dot(stocks[stocks.apply(lambda x: x < x.quantile(0.05))].mean().values)
cvar95_hist

-0.03145432615142683

So, the Conditional-Value-at-Risk after the VaR exceeds is 3.14% with 5% probability of loss. The results can be summarized in the table below:

|            | VaR95% | CVaR95% |
|------------|--------|---------|
| Historical |  2.15% |  3.14%  |

> Using expected mean, covariance matrix and parametric method, calculate VaR95% and CVaR95%

The second method that can be used to calculate the VaR and CVaR is parametric method. Here, we calculate the mean and standard deviation of the returns and calculate the `norm` by generating the normal distribution from these values and the desired level of confidence. This gives the VaR using the formula

VaR95 = norm.ppf(0.95)*std - mean

and CVaR using the formula

CVaR95 = (1-0.95)^-1 * norm.pdf(norm.ppf(1-0.95))*std - mean

Source: https://quantatrisk.com/2016/12/08/conditional-value-at-risk-normal-student-t-var-model-python/

In [22]:
# mean of the returns of the stocks
stocks.mean()

AAPL    0.000574
IBM     0.001022
GOOG    0.000236
BP      0.001210
XOM     0.000820
COST    0.000121
GS      0.001406
dtype: float64

To get the mean for the portfolio we need to multiply this with the stock weights.

In [23]:
# array of means of stock returns
stocks.mean().values

array([0.00057355, 0.00102234, 0.00023642, 0.0012103 , 0.0008201 ,
       0.00012059, 0.00140576])

In [24]:
# array of values of stock weights in our portfolio
port.values[0]

array([0.15, 0.2 , 0.2 , 0.15, 0.1 , 0.15, 0.05])

In [25]:
# dot product of the above two gives the expected mean for returns for the portfolio
mean_port = port.values[0].dot(stocks.mean().values)
mean_port

0.0006897150610100082

In [26]:
# covariance matrix of the stocks
cov_matrix = stocks.cov()
cov_matrix

,AAPL,IBM,GOOG,BP,XOM,COST,GS
AAPL,0.000217,0.000056,0.000087,0.000079,0.000048,0.000052,0.000090
IBM,0.000056,0.000155,0.000053,0.000095,0.000065,0.000024,0.000089
GOOG,0.000087,0.000053,0.000157,0.000049,0.000029,0.000049,0.000065
BP,0.000079,0.000095,0.000049,0.000343,0.000153,0.000027,0.000168
XOM,0.000048,0.000065,0.000029,0.000153,0.000146,0.000021,0.000085
COST,0.000052,0.000024,0.000049,0.000027,0.000021,0.000122,0.000046
GS,0.000090,0.000089,0.000065,0.000168,0.000085,0.000046,0.000286


To get the std for the portfolio we need to multiply this with the stock weights.

In [27]:
# values of covariance of stock returns
cov_matrix.values

array([[2.17006162e-04, 5.59911249e-05, 8.74064932e-05, 7.92364963e-05,
        4.84425721e-05, 5.15074993e-05, 8.98616572e-05],
       [5.59911249e-05, 1.54862010e-04, 5.29871619e-05, 9.52117679e-05,
        6.49422577e-05, 2.39766048e-05, 8.94850475e-05],
       [8.74064932e-05, 5.29871619e-05, 1.56958485e-04, 4.87019617e-05,
        2.93159135e-05, 4.90976612e-05, 6.53544026e-05],
       [7.92364963e-05, 9.52117679e-05, 4.87019617e-05, 3.43460461e-04,
        1.52618496e-04, 2.65717162e-05, 1.67903311e-04],
       [4.84425721e-05, 6.49422577e-05, 2.93159135e-05, 1.52618496e-04,
        1.45502012e-04, 2.13323189e-05, 8.46586242e-05],
       [5.15074993e-05, 2.39766048e-05, 4.90976612e-05, 2.65717162e-05,
        2.13323189e-05, 1.22471562e-04, 4.62853460e-05],
       [8.98616572e-05, 8.94850475e-05, 6.53544026e-05, 1.67903311e-04,
        8.46586242e-05, 4.62853460e-05, 2.85757711e-04]])

In [28]:
# array of values of stock weights in our portfolio
port.values[0]

array([0.15, 0.2 , 0.2 , 0.15, 0.1 , 0.15, 0.05])

In [29]:
# dot product of the above two gives the standard deviation for returns for the portfolio
# reference from https://www.interviewqs.com/blog/value-at-risk
std_port = np.sqrt(port.values[0].dot(cov_matrix.values).dot(port.values[0]))
std_port

0.009049636381388665

Using the `scipy`'s `norm` we can get the desired values.

In [30]:
# import the library
from scipy.stats import norm

The mean and std of the portfolio are 0.0689% and 0.009, respectively.

In [31]:
# using the formulas above, we can calculate the VaR
var95_param = norm.ppf(0.95)*std_port - mean_port
var95_param

0.014195612163509135

In [32]:
# using the formulas above, we can calculate the CVaR
cvar95_param = (1-0.95)**-1 * norm.pdf(norm.ppf(1-0.95))*std_port - mean_port
cvar95_param

0.017977085806165548

So, the Value-at-Risk using parametric method assuming a normal distribution is 1.42% and Conditional-Value-at-Risk after the VaR exceeds is 1.79% with 5% probability of loss. The results can be summarized in the table below:

|            | VaR95% | CVaR95% |
|------------|--------|---------|
| Historical |  2.15% |  3.14%  |
| Parametric |  1.42% |  1.79%  |

> Assume you can change weights, allow shorting but no leverage (i.e. sum of weights equal 100%), and rebalance monthly. What is the optimal portfolio holding by end of each month till end of 2016

Here, we can do something similar as above but lets say we calculate the returns at the end of every month and maximise our portfolio returns by adapting weights accordingly. If the returns of a particular stock are on a declining trend, we can use volatility to decrease the weights for that stock, and vice-versa.

#### 5. Assume you have a Python project, which source code is under a git repo folder “my-python-project”. Write a program/script to produce the following statistics of this folder:

  - How many python files
  - How many lines of code in total, how many lines of comment line (empty line doesn’t
count)
  - How many functions is defined in total
  - How many lines of changes from the current version against HEAD~3
  - Total folder size (in MB) per each of the subfolder (down to 2 level depth)

There are some readily available tools and packages that can be used to answer these. For the number of python files and lines of code, `pygount` can be used. For the folder sizes, `os` python library can be used. This can also be used to answer the first question. Additionally, there are some online tools available which can summarize a GitHub project. Lines changed between current version against HEAD~3 can be found out using `git`'s `log` or `diff` commands. For the number of functions, am sure there should be a package/library which we can use in our source code.

#### 6. In a text file, give me total number of appearance of “date” within the text file. The date format can appears in either one (or multiple) formats shown below:

  - YYYY/MM/DD
  - MM/DD/YYYY
  - DD/MM/YYYY
  - DD (Jan/Feb/Mar/Apr/May/Jun/Jul/Aug/Sept/Oct/Nov/Dec) YYYY

This is a perculiar problem to solve as the data is corrupted. Though there's an easy way address this by using the `pandas` library's `to_datetime()` function. It will convert almost all formats of datetime into a standard format. However, there are a few corner cases which might be good to highlight because of the nature of the data, for example, there is no way to differentiate between MM/DD/YYYY and DD/MM/YYYY for dates less than 13. The library is smart enough to identify month if the date is more than 13 otherwise it will consider it as default format. This is a drawback of the data which cannot be solved even by humans. Secondly, another assumption we make is that the text file contains each date in a new line. We will read the file to a `pandas DataFrame` and then use `to_datetime()` method to convert the dates into a standard format.

In [33]:
# import the python library
import pandas as pd

# import the text file with dates to dataframe
df = pd.read_csv('dates.txt', sep='\n', names=['date'])

In [34]:
# let's see the content of a sample text file
with open('dates.txt') as f:
    print(f.read())

2021/07/25
08/12/2021
24/05/2021
random word
03/25/2021
15 Aug 2021



In [35]:
# convert the dates to datetime format
pd.to_datetime(df.date, errors='coerce')

0   2021-07-25
1   2021-08-12
2   2021-05-24
3          NaT
4   2021-03-25
5   2021-08-15
Name: date, dtype: datetime64[ns]

In [36]:
# Finally, the number of dates in the text file
pd.to_datetime(df.date.dropna(), errors='coerce').count()

5

There are 5 dates in the sample file we used here. Ok, in this case since we assumed that each date was on a separate line we could have just opened the file and counted the number of lines that have some date format. But this code will work for other cases as well, say the dates are separated by other delimiters. In case it is a full text file like an essay, then when reading the file we have use a logic to find the dates first, store them and then convert to dataframe and subsequently to datetime format. There are multiple ways to answer this question depending on use case.